In [1]:
import gym
import stable_baselines3
from stable_baselines3 import PPO, DDPG, A2C

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random
import math
from functools import partial
from scipy.stats import gamma

import json
import os
import sys
import time

import fair
from fair.RCPs import rcp26, rcp45, rcp60, rcp85
from fair.SSPs import ssp370, ssp245, ssp585

In [2]:
current_year = 2022
start_index = current_year - ssp245.Emissions.year[0] # 2022 - 1765
size = ssp245.Emissions.year.size
print(start_index)
print(size)

257
736


In [3]:
fair.__version__

'1.6.4'

In [4]:

      
        
#Implementation of S, Y and Y_cost
#S is renewable energy stock, Y is GDP, Y_cost is cost of climate change


# Economic dimension
Y = np.zeros(size) #global GDP
Y_cost = np.zeros(size) #cost of climate change

S = np.zeros(size) #renewable knowledge stock
S[:start_index] = 5e11 #GJ
Y[:start_index] = 9e13 #USD/a
Y_cost[:start_index] = 1e11 #USD/a

S[start_index] = 5e11 #GJ
Y[start_index] = 9.2e13 #USD/a
Y_cost[start_index] = 1e11 #USD/a

beta = 0.03 # estimated GDP growth rate over the year

# used to calculate energy stock
epsilon = 147.0 # USD/GJ
rho = 2.0 # 1, 
sigma = 4e12 # GJ, 
tau_S = 65.0 # yr,
        
"""gamma = 1 / ( 1+(S[self.t-1]/sigma)**rho )
Y[self.t] = (1+beta) * Y[self.t-1] # calculate using assumed GDP growth rate
Y_cost[self.t] = ((10/5*T[self.t]-2)/100)*Y[self.t] # assumed effect on GDP
S[self.t] = S[self.t-1] + ( (1-gamma)*Y[self.t-1]/epsilon - S[self.t-1]/tau_S )
# fail if temperature error
if math.isnan(T[self.t]):
    done = True


# determine economic and other parameters to feed into loss function
if self.multigas:
    cur_emit = self.emissions[:,1][self.t] + self.emissions[:,2][self.t]
    cur_conc = C[:,0][self.t]
else:
    cur_emit = self.emissions[self.t]
    cur_conc = C[self.t]

cur_fease = cur_emit - self.ssp_370[self.t-1]
cur_GDP = Y[self.t] - Y_cost[self.t]

#compute the reward

# GDP SEEMS WRONG- what is / was PIB. Also look at PIB. 
# seem to be computing percent change in GDP, not GDP itself yet this is not how reward function is treating it. 
# RESOLVE SUS-NESS

reward = self.reward_func(self.state, T[self.t], self.t, cur_emit, cur_conc, cur_GDP, Y_cost[self.t], cur_fease)


"""

beta

0.03

In [5]:
# Economic dimension
Y = np.zeros(size) #global GDP
Y_cost = np.zeros(size) #cost of climate change

S = np.zeros(size) #renewable knowledge stock
S[start_index-1] = 5e11 #GJ
Y[start_index-1] = 9e13 #USD/a
Y_cost[start_index-1] = 100*1e9 #USD/a

S[start_index] = 5e11 #GJ
Y[start_index] = 9.2e13 #USD/a
Y_cost[start_index] = 100*1e9 #USD/a

beta = 0.03 # estimated GDP growth rate over the year
epsilon = 147.0 # USD/GJ
rho = 2.0 # 1
sigma = 4e12 # GJ
tau_S = 65.0 # yr


In [145]:
#### Reward function options ####
def simple_reward(state, cur_temp, year, cur_emit, cur_conc, cur_GDP, GDP_cost, cur_fease):
# positive reward for temp decrease
# negative cliff if warming exceeds 2º
    if cur_temp > 2:
        return -100
    return (state[0] - cur_temp) # punish if old temp is lower than new temp

def temp_reward(state, cur_temp, year, cur_emit, cur_conc, cur_GDP, GDP_cost, cur_fease):
# positive reward for temp under 1.5 goal
    if cur_temp > 2:
        return -100
    return 1.5 - cur_temp

def conc_reward(state, cur_temp, year, cur_emit, cur_conc, cur_GDP, GDP_cost, cur_fease):
    # positive reward for decreased concentration
    if cur_temp > 2:
        return -100
    return state[2] - cur_conc

def carbon_cost_reward(state, cur_temp, year, cur_emit, cur_conc, cur_GDP, GDP_cost, cur_fease):
    # impose a cost for each GtC emitted
    if cur_temp > 2:
        return -100
    return -cur_emit

def carbon_cost_GDP_reward(state, cur_temp, year, cur_emit, cur_conc, cur_GDP, GDP_cost, cur_fease):
    reward = 0
    if cur_temp > 2:
        reward += -1000
    if cur_fease < 0:
        reward += -100
    if cur_fease > 0:
        reward += 10*cur_fease
    reward += cur_GDP/1e11
    return reward

def temp_emit_reward(state, cur_temp, year, cur_emit, cur_conc, cur_GDP, GDP_cost, cur_fease):
    # positive reward for keeping the temp under 1.5
    # negative reward for amount of emissions reduction
    # positive cliff for success at the end of the trial
    # w could indicate cost
    if cur_temp > 2:
        return -100
    if year==2100 and temp<=1.5:
        return 1000
    temp = 10*(state[0] - cur_temp)
    emit_decrease = state[1] - cur_emit # decrease in emissions
    if emit_decrease>0:
        return temp + emit_decrease
    return temp


def temp_emit_diff_reward(state, cur_temp, year, cur_emit, cur_conc, cur_GDP, GDP_cost, cur_fease):
    # positive reward for keeping the temp under 1.5
    # negative reward for amount of emissions reduction
    # (reduction compared to projected amount for that year)
    # positive cliff for success at the end of the trial
    # w could indicate cost of emissions
    if cur_temp > 2:
        return -100
    if t==79 and temp <=1.5:
        return 100
    curval = t*0.6 + 36
    temp = 10*(state[0] - cur_temp)
    emit = curval - cur_emit
    if cur_emit < curval:
        return temp - emit
    return temp

# Defining Environment

In [146]:
"""
Create an Environment class and extend following functions:

1. Constructor- initialize continuous action space and observation space, select reward function, 
    define time advancement function from FaIR. 

2. update_state takes in C, F, T which corresponds to emission data and changes params of self.state and self.t

3. reset() sets all emissions to baseline from provided SSP scenario

4. step(action) takes in action and computes forward emissions data from environment. updates state with what is computed and returns reward

5. render() plots all the relevant climate info. 


""" 



class Simulator(gym.Env):
    def __init__(self, action_space=36, reward_mode="temp_emit_reward", climate_params={},
                 forcing=False, multigas=True, scenario='ssp245', verbose=1, current_year=2022, end_year=2100):
        
        # action space for the environment,
        # the amount to increase or decrease emissions by
        self.action_space = gym.spaces.Box(
            np.array([-action_space]).astype(np.float32),
            np.array([+action_space]).astype(np.float32),
        )
        
        # state space, [temperature, carbon emissions, carbon concentration, radiative forcing]
        if not multigas:
            self.observation_space = gym.spaces.Box(
                np.array([-100, -100, 0, -100]).astype(np.float32),
                np.array([100, 100, 5000, 100]).astype(np.float32),
            )
        else:
            self.observation_space = gym.spaces.Box( # both length 16
                np.array([-100, -100, 0, -50, -50, -50, -50, -50, -50, -50, -50,
                -50, -50, -50, -50, -50]).astype(np.float32),
                
                np.array([100, 100, 5000, 50, 50, 50, 50, 50, 50, 50, 50, 50,
                50, 50, 50, 50]).astype(np.float32),
            )
        
        # specify the reward function to use
        
        self.reward_func = eval(reward_mode)

        # setup climate parameters for FaIR model
        
        if forcing:
            climate_params['F_solar'] = 0.1 * np.sin(2 * np.pi * np.arange(736) / 11.5)
            climate_params['F_volcanic'] = -gamma.rvs(0.2, size=736, random_state=14) # samples from gamma probability distribution
        climate_params["useMultigas"] = multigas
        self.forward_func = partial(fair.forward.fair_scm, **climate_params)
        
        self.multigas = multigas
        self.scenario = eval(scenario)
        self.verbose = verbose
        self.forcing = forcing
        self.state = None
        
        # setup time bounding
        self.start_index = current_year - self.scenario.Emissions.year[0] # 2022 - 1765
        self.end_timestep = end_year - self.scenario.Emissions.year[0] # # 2100 - 1765
    
    
    def current_year(self):
        return(self.scenario.Emissions.year[self.t])
        
    
    
    def reset(self):
        base_emissions = self.scenario.Emissions.emissions
        ssp_370_emissions = ssp370.Emissions.emissions
        if not self.multigas:
            base_emissions = np.array([x[1]+x[2] for x in base_emissions])
            ssp_370_emissions = np.array([x[1]+x[2] for x in ssp_370_emissions])
        
        # 80 year time horizon, meet goals by 2100
        self.emissions = base_emissions
        # 2022 estimate of GtC of carbon emissions
        # 2022 is the 257th year in the ssp scenario
        self.ssp_370 = ssp_370_emissions #for feasibility testing later
    
        self.t = self.start_index - 1
        # initial state
        C, F, T = self.forward_func( #forward func computes from FaIR climate simulator, returns world
            emissions=self.emissions,
        )
        
        self.update_state(C,F,T)
        return(self.state)
    

        
    def update_state(self, C, F, T):
        
        # state is always [temp, emission, concentration, forcing] all as scalars in given year
        if self.multigas:
            concentration = C[:,0][self.t]
            forcing = np.sum(F,axis=1)[self.t]
            emissions = self.emissions[:,1][self.t] + self.emissions[:, 2][self.t]
        else:
            concentration = C[self.t]
            forcing = F[self.t]
            emissions = self.emissions[self.t]
        
        self.state = [T[self.t], emissions, concentration, forcing]
        self.t += 1
        
    
    def step(self, action):
        
        #should not call step without calling reset()
        if(not self.state): #has not been initialized
            self.reset()
        
        done = False
        
        # change emissions by the action amount
        if not self.multigas:
            # change time step to be previous time step + delta E from action (negative). Floor is zero
            self.emissions[self.t] = max(self.emissions[self.t-1] + action[0], 0)
        else:
            self.emissions[:,1][self.t] = max(self.emissions[:,1][self.t-1] + action[0]*.9, 0)
            self.emissions[:,2][self.t] = max(self.emissions[:,2][self.t-1] + action[0]*.1, 0)
        # run FaIR simulator
        
        C, F, T = self.forward_func(
            emissions=self.emissions
        )
        
        # update the state and info; keep old_state for reward later
        old_state = self.state
        self.update_state(C, F, T)
        
        # determine economic and climate parameters to feed into loss function
        # Implementation of S, Y and Y_cost
        #S is renewable energy stock, Y is GDP, Y_cost is cost of climate change
        
        gamma = 1 / ( 1+(S[self.t-1]/sigma)**rho )
        Y[self.t] = (1 + beta) * Y[self.t-1] #
        Y_cost[self.t] = ((10/5*T[self.t]-2)/100) * Y[self.t]
        S[self.t] = S[self.t-1] + ( (1-gamma)*Y[self.t-1]/epsilon - S[self.t-1]/tau_S )
        # fail if temperature error
        if math.isnan(T[self.t]):
            done = True
            
        
        #compute the reward
        
        reward = self.reward_func(old_state, cur_temp, self.current_year(), cur_emit, cur_conc, cur_GDP, Y_cost[self.t], cur_fease)
        
        # end the trial once 2100 is reached
        if self.t == self.end_timestep or self.state[0] > 4 or self.state[0] < 0: # only runs to 2100
            done = True
    
    
        return self.state, reward, done, {}
    
    
    def render(self, mode="human"):
        # print the state
        print(f"Current Year: {self.current_year()}")
        print(f'Temperature anomaly: {self.state[0]}ºC')
        print(f'CO2 emissions: {self.state[1]} GtC')
        print(f'CO2 concentration: {self.state[2]} ppm')
        print(f'Radiative forcing: {self.state[3:]}')
        




In [147]:
args = {
    'name' : "temp_emit_a2c",
    'directory' : "baby_steps", #the path in which the run will be saved
    'action_space' : 36,
    "seed": 158,
    'reward_mode' : "temp_emit_diff_reward",
    'forcing' : True,
    'multigas' : False,
    'scenario' : "ssp245",
    'algorithm' : "a2c",
    'learning_rate' : 2.1e-05,
    
    'gamma' : 0.9, 
    'device' : 'cpu',
    'iterations' : 300, 
    "n_steps" : 5,
    'verbose' : 1,
    'log_freq' : 20,
    
}

climate = {
    "a" : np.array([0.2173, 0.2240, 0.2824, 0.2763]),
    "tau" : np.array([1000000, 394.4, 36.54, 4.303]),
    "r0" : 32.40,
    "rc" : 0.019,
    "rt" : 4.165,
}




In [148]:
def test_env(env, climate_args, iters):    
    env.reset()
    print("Start Year: ", env.current_year())
    for i in range(iters):
        action = env.action_space.sample()
        state, reward, done, _ = env.step(action)
        if(done):
            print("\n\nSimulation concluded at iteration: ", i)
            print(env.t, env.current_year())
            env.render()
            env.reset()
    print("\nFinished Testing")
    env.render()
    env.reset()

env = Simulator(args['action_space'], reward_mode=args['reward_mode'], climate_params=climate, 
        forcing=args['forcing'], multigas=args['multigas'], scenario=args['scenario'], verbose=args['verbose'])

#test_env(env, climate, 100)

env.reset()

[0.9595145294408387, 11.169595587494776, 414.0989964569708, 2.1328449551182502]

In [154]:
emissions = np.array([x[1] + x[2] for x in ssp370.Emissions.emissions])

C, F, T = env.forward_func(emissions = emissions[:370])
C.shape, F.shape, T.shape

((370,), (370,), (370,))

In [96]:
from stable_baselines3.common.logger import configure

In [97]:
args = {
    'name' : "temp_emit_a2c",
    'directory' : "fair_1.6_basic", #the path in which the run will be saved
    'action_space' : 45,
    "seed": 158,
    'reward_mode' : "carbon_cost_GDP_reward",
    'forcing' : True,
    'multigas' : False,
    'scenario' : "ssp245",
    'algorithm' : "a2c",
    'learning_rate' : 2.1e-04,
    
    'gamma' : 0.9, 
    'device' : 'cpu',
    'iterations' : 1000, 
    "n_steps" : 5,
    'verbose' : 1,
    'log_freq' : 25,
    
}

climate = {
    "a" : np.array([0.2173, 0.2240, 0.2824, 0.2763]),
    "tau" : np.array([1000000, 394.4, 36.54, 4.303]),
    "r0" : 32.40,
    "rc" : 0.019,
    "rt" : 4.165,
}


In [98]:
def setup_dirs(args):
    save_path = os.path.join("outputs", args['directory'])
    dirs = ['plots', 'logs', 'saved_models', 'evals']
    for direc in dirs:
        path = os.path.join(save_path, direc)
        if not os.path.exists(path):
            os.makedirs(path)
    
    # save args configs
    with open(os.path.join(save_path, 'config.txt'), 'w') as file:
        json.dump(args, file, indent=2)
    
    # Set random seeds for reproducibility
    random.seed(args['seed'])
    np.random.seed(args['seed'])
    torch.manual_seed(args['seed'])
    
    return save_path

def create_model(args=args, climate_args=climate, env_type=Simulator):
    save_path = setup_dirs(args)
    
    env = env_type(args['action_space'], reward_mode=args['reward_mode'], climate_params=climate_args, 
        forcing=args['forcing'], multigas=args['multigas'], scenario=args['scenario'], verbose=args['verbose'])
    
    env.reset()
    
    model_builder = eval(args['algorithm'].upper()) # import from stable baselines
    model = model_builder(
        policy="MlpPolicy",
        env=env,
        learning_rate = args['learning_rate'],
        n_steps = args['n_steps'],
        gamma= args['gamma'],
        verbose=args['verbose'],
        tensorboard_log=os.path.join(save_path, 'logs'),
    )

    model.set_logger(configure(
        os.path.join(save_path, 'logs'),
        ["csv", "tensorboard"]
    ))
    
    return model, save_path

def train_model(model, save_path, timesteps=args['iterations'], log_f = args['log_freq']):
    
    model.learn(
        total_timesteps=timesteps,
        log_interval=log_f,
        eval_log_path=os.path.join(os.path.join(save_path, 'eval'), args['name']), #eval_freq=20,
    )
    
    saved_path = os.path.join(os.path.join(save_path, 'saved_models'), args['name'])
    model.save(saved_path)
    
    return saved_path, model


In [99]:
model, save_path = create_model(args, climate)
model, save_path

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


(<stable_baselines3.a2c.a2c.A2C at 0x19d2ccf40>, 'outputs/fair_1.6_basic')

In [100]:
train_model(model, save_path)

('outputs/fair_1.6_basic/saved_models/temp_emit_a2c',
 <stable_baselines3.a2c.a2c.A2C at 0x19d2ccf40>)

# Testing and Evaluating Model

In [101]:
## labels for plots ##
VARS = [
    "Year",
    "Temperature anomaly (ºC)",
    "Emissions (GtC)",
    "CO2 Concentration (ppm)",
    "Radiative forcing (W / m^2)",
    "Reward"
]

MULTIGAS_VARS = [
    "Temperature anomaly (ºC)",
    "CO2 Emissions (GtC)",
    "CO2 Concentration (ppm)",
    "CO2 forcing (W m^-2)",
    "CH4 forcing (W m^-2)",
    "N2O forcing (W m^-2)",
    "All other well-mixed GHGs forcing (W m^-2)",
    "Tropospheric O3 forcing (W m^-2)",
    "Stratospheric O3 forcing (W m^-2)",
    "Stratospheric water vapour from CH4 oxidation forcing (W m^-2)",
    "Contrails forcing (W m^-2)",
    "Aerosols forcing (W m^-2)",
    "Black carbon on snow forcing (W m^-2)",
    "Land use change forcing (W m^-2)",
    "Volcanic forcing (W m^-2)",
    "Solar forcing (W m^-2)",
    "Reward ",
]

In [102]:
#### Training code ####

def evaluate_model(test_iters=100, env_class=Simulator, args=args, climate_args=climate):
    save_path = os.path.join("outputs", args['directory'])
    model_path = os.path.join(os.path.join(save_path, 'saved_models'), args['name'])
    
    env = env_class(args['action_space'], reward_mode=args['reward_mode'], climate_params=climate_args, 
        forcing=args['forcing'], multigas=args['multigas'], scenario=args['scenario'], verbose=args['verbose'])
    
    model = eval(args['algorithm'].upper()).load(model_path)
    progress = pd.read_csv(os.path.join(save_path, "logs/progress.csv"))
    
    obs = env.reset()
    vals = []
    for i in range(test_iters):
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, _ = env.step(action)
        vals.append([env.current_year()] + obs + [reward]) # adds reward to obs list [a,b] + [c] = [a,b,c]
        if((i-1)%25==0):
            print("Iteration number: ", i)
        
        if done:
            print("\n\nSimulation concluded at iteration: ", i)
            print(env.t, env.current_year())
            env.render()
            env.reset()
            break;
    if(env.close is not None):
        env.close()
    
    
    
    
    
    
    return model, pd.DataFrame(data=vals, columns=VARS)
    

 

In [103]:
model, vals = evaluate_model()

Iteration number:  1
Iteration number:  26
Iteration number:  51
Iteration number:  76


Simulation concluded at iteration:  77
335 2100
Current Year: 2100
Temperature anomaly: 1.153359711631791ºC
CO2 emissions: 0.0 GtC
CO2 concentration: 409.00208681961317 ppm
Radiative forcing: [2.0665565216074375]


In [104]:
vals

,Year,Temperature anomaly (ºC),Emissions (GtC),CO2 Concentration (ppm),Radiative forcing (W / m^2),Reward
0,2023,0.975008,10.778328,416.416315,2.162714,-16.225017
1,2024,0.990158,10.386993,418.565154,2.190263,-22.607654
2,2025,1.004858,9.995577,420.562490,2.215743,-28.999186
3,2026,1.019039,9.604067,422.418653,2.239314,-35.398301
4,2027,1.032653,9.212447,424.140238,2.261084,-41.804031
...,...,...,...,...,...,...
95,2040,1.153071,4.092651,435.339011,2.400572,-120.316745
96,2041,1.158027,3.696829,435.386152,2.401151,-126.114105
97,2042,1.162382,3.300550,435.320468,2.400344,-131.910137
98,2043,1.166140,2.903825,435.142539,2.398156,-137.363985


In [106]:
# Output useful plots
def make_plots(vals, save_path):
    for col in vals.columns[1:]:
        xs = vals['Year']
        ys = vals[col]
        plt.plot(xs, ys)
        plt.ylabel(col)
        plt.xlabel('Year')
        plt.savefig(os.path.join(save_path, 'plots', col))
        plt.clf()
    
    vals.to_csv(os.path.join(save_path, 'saved_models','model_output.csv'))
    #with open(os.path.join(save_path, 'saved_models','model_output.csv'), 'w') as f:
        #json.dump(excel, f, indent=None, separators=None)make_plots(vals, args, save_path)


In [76]:
make_plots(vals, save_path)

<Figure size 640x480 with 0 Axes>

In [77]:
pd.read_csv(os.path.join(save_path, 'saved_models','model_output.csv'))

,Unnamed: 0,Year,Temp_Anom,Emissions,Concentration,Forcing,Reward
0,0,2023,0.975008,10.778328,416.416315,2.162714,-16.225017
1,1,2024,0.990158,10.386993,418.565154,2.190263,-22.607654
2,2,2025,1.004858,9.995577,420.562490,2.215743,-28.999186
3,3,2026,1.019039,9.604067,422.418653,2.239314,-35.398301
4,4,2027,1.032653,9.212447,424.140238,2.261084,-41.804031
...,...,...,...,...,...,...,...
95,95,2040,1.153071,4.092651,435.339011,2.400572,-120.316745
96,96,2041,1.158027,3.696829,435.386152,2.401151,-126.114105
97,97,2042,1.162382,3.300550,435.320468,2.400344,-131.910137
98,98,2043,1.166140,2.903825,435.142539,2.398156,-137.363985
